In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import h5py
import os
from tqdm import trange
import scipy.ndimage as ndimage
from functools import reduce


In [ ]:
data_path_1 ="/nfs/data1/chuyu/data/20230227_155813/"
data_path_2 ="/nfs/data1/chuyu/data/20230227_165253/"
data_path_3 ="/nfs/data1/chuyu/data/20230227_175948/"

In [ ]:
file_name = "neuron_spatial_info_15_47_chamber_geometry_20230227_155813_sigma2_n100_A_dF.h5"
data = h5py.File(os.path.join(data_path_1, file_name), "r")
place_map_all_1= np.array(data["place_map_all"])
specificity_1= np.array(data["specificity"])
place_cell_index_1= np.array(data["place_cell_index"])-1
valid_neurons_1 = np.array(data["valid_neurons"])-1
data.close()

In [ ]:
file_name = "neuron_spatial_info_0_64_chamber_geometry_20230227_165253_sigma2_n100_A_dF.h5"
data = h5py.File(os.path.join(data_path_2, file_name), "r")
place_map_all_2= np.array(data["place_map_all"])
specificity_2= np.array(data["specificity"])
valid_neurons_2 = np.array(data["valid_neurons"])-1
data.close()

In [ ]:
file_name = "neuron_spatial_info_0_61_chamber_geometry_20230227_175948_sigma2_n100_A_dF.h5"
data = h5py.File(os.path.join(data_path_3, file_name), "r")
place_map_all_3= np.array(data["place_map_all"])
specificity_3= np.array(data["specificity"])
valid_neurons_3 = np.array(data["valid_neurons"])-1
data.close()

In [ ]:
file_name = "whether_tel.h5"
data = h5py.File(os.path.join(data_path_1, file_name), "r")
whether_tel= np.array(data["whether_tel"])
data.close()

In [ ]:
candidate_neurons = reduce(np.intersect1d, (np.where(whether_tel)[0], valid_neurons_1, valid_neurons_2, valid_neurons_3, np.where((specificity_1>0.01))[0], np.where((specificity_2>0.01))[0], np.where((specificity_3>0.01))[0]))

In [ ]:
mask_valid = np.isfinite(place_map_all_1[1, :,:])

plt.figure()
plt.imshow(mask_valid)
edge_roi = ndimage.morphology.binary_dilation(mask_valid)*1 - mask_valid*1
plt.figure()
plt.imshow(edge_roi)
edge_pixels = np.where(edge_roi==1)
edge_pixels_array_1 = np.vstack([edge_pixels[1], edge_pixels[0]]).T
plt.figure()
plt.scatter(edge_pixels_array_1[:,0], edge_pixels_array_1[:,1])

In [ ]:
mask_valid = np.isfinite(place_map_all_2[1, :,:])

plt.figure()
plt.imshow(mask_valid)
edge_roi = ndimage.morphology.binary_dilation(mask_valid)*1 - mask_valid*1
plt.figure()
plt.imshow(edge_roi)
edge_pixels = np.where(edge_roi==1)
edge_pixels_array_2 = np.vstack([edge_pixels[1], edge_pixels[0]]).T
plt.figure()
plt.scatter(edge_pixels_array_2[:,0], edge_pixels_array_2[:,1])

In [ ]:
x_locs, counts = np.unique(edge_pixels[1], return_counts=True)
counts_order = (-counts).argsort()
counts_ranks = counts_order.argsort()
mid_wall_loc = x_locs[counts_order[2]]
mid_wall_loc

In [ ]:
mask_valid = np.isfinite(place_map_all_3[1, :,:])

plt.figure()
plt.imshow(mask_valid)
edge_roi = ndimage.morphology.binary_dilation(mask_valid)*1 - mask_valid*1
plt.figure()
plt.imshow(edge_roi)
edge_pixels = np.where(edge_roi==1)
edge_pixels_array_3 = np.vstack([edge_pixels[1], edge_pixels[0]]).T
plt.figure()
plt.scatter(edge_pixels_array_3[:,0], edge_pixels_array_3[:,1])

In [ ]:
from numba import njit


@njit
def normpdf(x, mean, sd):
    var = float(sd)**2
    denom = (2*math.pi*var)**.5
    num = math.exp(-(float(x)-float(mean))**2/(2*var))
    return num/denom


@njit
def normpdf_mean0(x, sd):
    var = float(sd)**2
    denom = (2*math.pi*var)**.5
    num = math.exp(-(float(x))**2/(2*var))
    return num/denom


@njit
def distance_from(points, target):
    nr_points = points.shape[0];
    distance = np.zeros(nr_points);
    
    for index in range(nr_points):
        distance[index] = norm_2(points[index,0]-target[0], points[index,1]-target[1]);

    return distance


@njit
def angle_from(points, target):
    nr_points =  points.shape[0];
    angle_all = np.zeros(nr_points);
    
    for index in range(nr_points):
        vector_1 = points[index,:]
        vector_2 = target
        vector_diff = vector_1 - vector_2
        complex = vector_diff[0] + vector_diff[1]*1j
        angle_all[index] = np.angle(complex)

    return angle_all;


@njit
def BVC_single(r, θ, d, Φ, σ_rad, σ_ang):
    g1 = normpdf(r, d, σ_rad)
    diff_angle = θ - Φ
    if diff_angle > np.pi:
        diff_angle = diff_angle -2*np.pi
    if diff_angle < - np.pi:
        diff_angle = diff_angle +2*np.pi
        
    g2 = normpdf_mean0(diff_angle, σ_ang)
    g = g1*g2
    return g

@njit
def norm_2(x, y):
    return np.sqrt(x**2+y**2)



@njit
def BVC(which_pixels, d, Φ, σ_rad, σ_ang, A, c, boundary_array):
    nr_pixel = which_pixels.shape[0]
    f_all = np.zeros(nr_pixel)
    for i_pixel in range(nr_pixel):
        xy = which_pixels[i_pixel,:]
        r_all = distance_from(boundary_array, xy);
        θ_all = angle_from(boundary_array, xy);
        f = A*sum([BVC_single(r_all[i], θ_all[i], d, Φ, σ_rad, σ_ang) for i in range(len(r_all))])
        f_all[i_pixel] = f
    return f_all + c 

In [ ]:
BVC_model_1 = lambda which_pixels, d, Φ, σ_rad, σ_ang, A, c : BVC(which_pixels, d, Φ, σ_rad, σ_ang, A, c, boundary_array= edge_pixels_array_1)
BVC_model_2 = lambda which_pixels, d, Φ, σ_rad, σ_ang, A, c : BVC(which_pixels, d, Φ, σ_rad, σ_ang, A, c, boundary_array= edge_pixels_array_2)
BVC_model_3 = lambda which_pixels, d, Φ, σ_rad, σ_ang, A, c : BVC(which_pixels, d, Φ, σ_rad, σ_ang, A, c, boundary_array= edge_pixels_array_3)

In [ ]:
p0 = [  3,  0, 1,   1, 3, 0]
bounds=([0, -np.pi, 0, 0, 0, -10], [10, np.pi, 10, 10, 100, 10])

# Check the model

In [ ]:
place_map_all = place_map_all_1
BVC_model = BVC_model_1
edge_pixels_array = edge_pixels_array_1

In [ ]:
which_map = place_map_all[1,:,:]
valid_pixels = np.where(np.isfinite(which_map))
valid_pixels_array = np.vstack([np.where(np.isfinite(which_map))[1], np.where(np.isfinite(which_map))[0]]).T

In [ ]:
valid_pixels[1].shape

In [ ]:
plt.scatter(edge_pixels_array[:,0], edge_pixels_array[:,1], s= 3, alpha=0.3)

plt.scatter(valid_pixels_array[:,0], valid_pixels_array[:,1], s= 3, alpha=0.3)

In [ ]:
plt.scatter(edge_pixels_array[:,0], edge_pixels_array[:,1], s= 3, alpha=0.3)

chosen_pixels = np.abs(valid_pixels_array[:,0] - mid_wall_loc) <20

plt.scatter(valid_pixels_array[chosen_pixels,0], valid_pixels_array[chosen_pixels,1], s= 3, alpha=0.3)

In [ ]:
d = 5
Φ = np.pi 
σ_rad = 1
σ_ang = 1
A = 3

c= 100

In [ ]:
f_all = BVC_model(valid_pixels_array, d, Φ, σ_rad, σ_ang, A, c)
vector_response = np.zeros_like(which_map)
vector_response[:] = np.nan
vector_response[valid_pixels] =  f_all

In [ ]:
f_all

In [ ]:
plt.imshow(vector_response)

In [ ]:
plt.imshow(vector_response)
plt.title(f"Φ = {Φ}")

# Fit one cell

In [ ]:
which_neuron = 19655
which_map = np.copy(place_map_all[which_neuron]);
which_map = which_map/np.nanstd(which_map)
# which_map[which_map>0.8]=0.8
plt.imshow(which_map)
plt.colorbar()

In [ ]:
np.nanmax(which_map)

In [ ]:
xdata = valid_pixels_array
ydata = which_map[valid_pixels];

In [ ]:
valid_pixels

In [ ]:
popt, pcov = curve_fit(BVC_model, xdata, ydata, p0 = p0, bounds = bounds, ftol=0.001)
d , Φ, σ_rad, σ_ang, A, c = popt

ypredict = BVC_model(xdata, d, Φ, σ_rad, σ_ang, A, c)
r = ydata - ypredict
chisq = np.sum((r)**2)
chisq

In [ ]:
popt

In [ ]:
vector_response = np.zeros_like(which_map)
vector_response[:] = np.nan
vector_response[valid_pixels] =  ypredict
plt.imshow(vector_response)
plt.colorbar()

# Fit all cells

In [ ]:
import tqdm

In [ ]:
BVC_models = [BVC_model_1, BVC_model_2, BVC_model_3]
place_map_all_all = [place_map_all_1, place_map_all_2, place_map_all_3]

In [ ]:
predict_map_all_all = []
popt_all_all = []
chisq_all_all = []
pass_neurons_all = []
failed_neurons_all = []

for i in range(3):
    BVC_model = BVC_models[i]
    place_map_all = place_map_all_all[i]
    

    nr_cells = place_map_all.shape[0]
    which_map = place_map_all[1,:,:]
    valid_pixels = np.where(np.isfinite(which_map))
    valid_pixels_array = np.vstack([np.where(np.isfinite(which_map))[1], np.where(np.isfinite(which_map))[0]]).T

    predict_map_all = np.zeros_like(place_map_all)
    popt_all = np.zeros((nr_cells, len(p0)))
    chisq_all = np.zeros(nr_cells)
    pass_neurons = []
    failed_neurons = []

    which_map = place_map_all[1]
    valid_pixels = np.where(np.isfinite(which_map))
    for which_neuron in tqdm.tqdm(candidate_neurons):
        try:

            which_map = place_map_all[which_neuron, :,:];
            which_map = which_map/np.nanstd(which_map)

            xdata = valid_pixels_array
            ydata = which_map[valid_pixels];
            popt, pcov = curve_fit(BVC_model, xdata, ydata, p0 = p0, bounds = bounds, ftol=0.001) # using 0.01, might be too small
            d , Φ, σ_rad, σ_ang, A, c = popt

            popt_all[which_neuron] = popt


            ypredict = BVC_model(xdata, d, Φ, σ_rad, σ_ang, A, c)
            r = ydata - ypredict
            chisq = np.sum((r)**2)
            chisq_all[which_neuron] = chisq

            vector_response = np.zeros_like(which_map)
            vector_response[valid_pixels] =  ypredict
            predict_map_all[which_neuron,:,:] = vector_response
            pass_neurons.append(which_neuron)


        except:
            failed_neurons.append(which_neuron)
            
            
            
    predict_map_all_all.append(predict_map_all)
    popt_all_all.append(popt_all)
    chisq_all_all.append(chisq_all)
    pass_neurons_all.append(pass_neurons)
    failed_neurons_all.append(failed_neurons)

In [ ]:
hf = h5py.File(os.path.join(data_path_1, "BVC_tel_newbound.h5"), 'w')
hf.create_dataset('candidate_neurons', data=candidate_neurons)
hf.create_dataset('popt_all_1', data=popt_all_all[0])
hf.create_dataset('popt_all_2', data=popt_all_all[1])
hf.create_dataset('popt_all_3', data=popt_all_all[2])
hf.close()

In [ ]:
file_name = "BVC_tel_newbound.h5"
data = h5py.File(os.path.join(data_path_1, file_name), "r")
popt_all_1= np.array(data["popt_all_1"])
popt_all_2= np.array(data["popt_all_2"])
popt_all_3= np.array(data["popt_all_3"])
data.close()

In [ ]:
popt_all_all = [popt_all_1 , popt_all_2, popt_all_3]

In [ ]:
nr_cells = place_map_all_all[0].shape[0]

In [ ]:
fit_neurons = np.intersect1d(np.where([any(popt_all_all[0][i,:]) for i in range(nr_cells)])[0], np.where([any(popt_all_all[1][i,:]) for i in range(nr_cells)])[0])

In [ ]:
predict_map_all_all = []
chisq_all_all = []


for i in range(3):
    BVC_model = BVC_models[i]
    place_map_all = place_map_all_all[i]
    popt_all = popt_all_all[i]

    nr_cells = place_map_all.shape[0]
    which_map = place_map_all[1,:,:]
    valid_pixels = np.where(np.isfinite(which_map))
    valid_pixels_array = np.vstack([np.where(np.isfinite(which_map))[1], np.where(np.isfinite(which_map))[0]]).T


    predict_map_all = np.zeros_like(place_map_all)
    chisq_all = np.zeros(nr_cells)

    for which_neuron in tqdm.tqdm(np.where(np.sum(popt_all != 0, axis=1) !=0)[0]):


        which_map = place_map_all[which_neuron, :,:];

        xdata = valid_pixels_array
        ydata = which_map[valid_pixels];
        d , Φ, σ_rad, σ_ang, A, c = popt_all[which_neuron]


        ypredict = BVC_model(xdata, d, Φ, σ_rad, σ_ang, A, c)
        r = ydata - ypredict
        chisq = np.sum((r)**2)
        chisq_all[which_neuron] = chisq

        vector_response = np.zeros_like(which_map)
        vector_response[valid_pixels] =  ypredict
        predict_map_all[which_neuron,:,:] = vector_response
            
            
    predict_map_all_all.append(predict_map_all)
    popt_all_all.append(popt_all)
    chisq_all_all.append(chisq_all)

In [ ]:
corr_all_all = []
for i in range(3):
    place_map_all = place_map_all_all[i]
    predict_map_all = predict_map_all_all[i]
    
    which_map = place_map_all[1,:,:]
    valid_pixels = np.where(np.isfinite(which_map))
    
    corr_all = np.zeros(nr_cells)

    for which_neuron in tqdm.tqdm(candidate_neurons):

        which_map_1 = place_map_all[which_neuron, :,:];
        which_map_2 = predict_map_all[which_neuron, :,:];

        corr_all[which_neuron] = np.corrcoef(which_map_1[valid_pixels], which_map_2[valid_pixels])[0,1]
        
    corr_all_all.append(corr_all)


In [ ]:
place_map_all = place_map_all_1
which_map = place_map_all[candidate_neurons[0],:,:]
valid_pixels_1 = np.where(np.isfinite(which_map))
valid_pixels_array_1 = np.vstack([np.where(np.isfinite(which_map))[1], np.where(np.isfinite(which_map))[0]]).T

place_map_all = place_map_all_2
which_map = place_map_all[candidate_neurons[0],:,:]
valid_pixels_2 = np.where(np.isfinite(which_map))
valid_pixels_array_2 = np.vstack([np.where(np.isfinite(which_map))[1], np.where(np.isfinite(which_map))[0]]).T

place_map_all = place_map_all_3
which_map = place_map_all[candidate_neurons[0],:,:]
valid_pixels_3 = np.where(np.isfinite(which_map))
valid_pixels_array_3 = np.vstack([np.where(np.isfinite(which_map))[1], np.where(np.isfinite(which_map))[0]]).T


which_map = place_map_all_1[candidate_neurons[0], :,:]*place_map_all_2[candidate_neurons[0], :,:]
valid_pixels = np.where(np.isfinite(which_map))
valid_pixels_array = np.vstack([np.where(np.isfinite(which_map))[1], np.where(np.isfinite(which_map))[0]]).T

corr_diff = np.zeros(nr_cells)

predict_map_12_all = np.zeros_like(place_map_all)
for which_neuron in tqdm.tqdm(candidate_neurons):
    map_1 = place_map_all_1[which_neuron, :,:];
    ydata_1 = map_1[valid_pixels];
    
    map_2 = place_map_all_2[which_neuron, :,:];
    ydata_2 = map_2[valid_pixels];
    
    xdata = valid_pixels_array
    
    popt = popt_all_all[0][which_neuron]
    if any(popt) == 0:
        continue
    d , Φ, σ_rad, σ_ang, A, c = popt

    ypredict_1 = BVC_model_1(xdata, d, Φ, σ_rad, σ_ang, A, c)
    ypredict_2 = BVC_model_2(xdata, d, Φ, σ_rad, σ_ang, A, c)
    
    vector_response = np.zeros_like(map_2)
    vector_response[valid_pixels] =  ypredict_2
    predict_map_12_all[which_neuron,:,:] = vector_response
    
    
    corr_diff[which_neuron] = np.corrcoef(ydata_2 - ydata_1, ypredict_2 - ypredict_1)[0,1]

In [ ]:
hf = h5py.File(os.path.join(data_path_1, "BVC_tel_newbound.h5"), 'r+')
hf.create_dataset('predict_map_12_all', data=predict_map_12_all)
hf.create_dataset('predict_map_all_1', data=predict_map_all_all[0])
hf.create_dataset('corr_diff', data=corr_diff)
hf.close()

In [ ]:
file_name = "BVC_tel_newbound.h5"
data = h5py.File(os.path.join(data_path_1, file_name), "r")
predict_map_12_all= np.array(data["predict_map_12_all"])
predict_map_all_1= np.array(data["predict_map_all_1"])
corr_diff= np.array(data["corr_diff"])
data.close()

# Selecting example neurons

In [ ]:
corr_threshold_global = 0.5
corr_threshold_diff = 0.4
valid_neurons = (corr_all_all[0][candidate_neurons] >corr_threshold_global)
valid_neurons_diff = (corr_diff >corr_threshold_diff)[candidate_neurons]

In [ ]:
Φ_1 = popt_all_all[0][candidate_neurons][:,1]
Φ_2 = popt_all_all[1][candidate_neurons][:,1]
Φ_3 = popt_all_all[2][candidate_neurons][:,1]

d_1 = popt_all_all[0][candidate_neurons][:,0]
d_2 = popt_all_all[1][candidate_neurons][:,0]
d_3 = popt_all_all[2][candidate_neurons][:,0];


σ_rad_1 = popt_all_all[0][candidate_neurons][:,2]
σ_rad_2 = popt_all_all[1][candidate_neurons][:,2]
σ_rad_3 = popt_all_all[2][candidate_neurons][:,2];


σ_ang_1 = popt_all_all[0][candidate_neurons][:,3]
σ_ang_2 = popt_all_all[1][candidate_neurons][:,3]
σ_ang_3 = popt_all_all[2][candidate_neurons][:,3];

In [ ]:
file_name = "NMF_merge.h5"
data = h5py.File(os.path.join(data_path_1, file_name), "r")
Z_all= np.array(data["Z_all"])
X_all= np.array(data["X_all"])
Y_all= np.array(data["Y_all"])
data.close()

In [ ]:
Φ_threshold = np.sqrt(2)/2
σ_rad_threshold = 6
σ_ang_threshold = 3

σ_pass = (σ_rad_1<σ_rad_threshold)*(σ_rad_3<σ_rad_threshold)*(σ_ang_1<σ_ang_threshold)*(σ_ang_3<σ_ang_threshold)


Φ_pass = (np.cos(Φ_1) < -Φ_threshold)*(np.cos(Φ_3) < -Φ_threshold)
pass_neurons_1 = candidate_neurons[np.where(Φ_pass*valid_neurons_diff)[0]]
Φ_pass = (np.cos(Φ_1) > Φ_threshold)*(np.cos(Φ_3) > Φ_threshold)
pass_neurons_2 = candidate_neurons[np.where(Φ_pass*valid_neurons_diff)[0]]


In [ ]:
pass_neurons_1 = pass_neurons_1[np.argsort(corr_diff[pass_neurons_1])][::-1]
pass_neurons_1

In [ ]:
pass_neurons_2 = pass_neurons_2[np.argsort(corr_diff[pass_neurons_2])][::-1]
pass_neurons_2

In [ ]:
plot_candidate = [51920, 17205, 39216, 57136, 19507]

In [ ]:
for which_neuron in plot_candidate:
    plt.figure(figsize=(20,5))
    plt.subplot(1,3,1)
    which_map = place_map_all_all[0][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)

    plt.subplot(1,3,2)
    which_map = place_map_all_all[1][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)
    
    plt.subplot(1,3,3)
    which_map = place_map_all_all[2][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)
    
    
    plt.figure(figsize=(20,5))
    plt.subplot(1,3,1)
    which_map = predict_map_all_all[0][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)

    plt.subplot(1,3,2)
    which_map = predict_map_all_all[1][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)
    
    plt.subplot(1,3,3)
    which_map = predict_map_all_all[2][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)

In [ ]:
file_name = "NMF_merge.h5"
data = h5py.File(os.path.join(data_path_1, file_name), "r")
Z_all= np.array(data["Z_all"])
X_all= np.array(data["X_all"])
Y_all= np.array(data["Y_all"])
data.close()

In [ ]:
invalid_neurons_diff = (corr_diff <0.2)[candidate_neurons]

In [ ]:
Φ_threshold = np.sqrt(2)/2
σ_rad_threshold = 6
σ_ang_threshold = 3

σ_pass = (σ_rad_1<σ_rad_threshold)*(σ_rad_3<σ_rad_threshold)*(σ_ang_1<σ_ang_threshold)*(σ_ang_3<σ_ang_threshold)


Φ_pass = (np.cos(Φ_1) < -Φ_threshold)*(np.cos(Φ_3) < -Φ_threshold)
no_pass_neurons_1 = candidate_neurons[np.where(Φ_pass*invalid_neurons_diff*valid_neurons)[0]]
Φ_pass = (np.cos(Φ_1) > Φ_threshold)*(np.cos(Φ_3) > Φ_threshold)
no_pass_neurons_2 = candidate_neurons[np.where(Φ_pass*invalid_neurons_diff*valid_neurons)[0]]


In [ ]:
no_pass_neurons_1 = np.intersect1d(no_pass_neurons_1[np.argsort(corr_all[no_pass_neurons_1])][::-1], fit_neurons)
no_pass_neurons_1

In [ ]:
no_pass_neurons_2 = np.intersect1d(no_pass_neurons_2[np.argsort(corr_all[no_pass_neurons_2])][::-1], fit_neurons)
no_pass_neurons_2

In [ ]:
plot_candidate_nonbvc = np.intersect1d([4616, 4646, 7670, 505, 3507, 5406, 7795, 15967, 5398], candidate_neurons)
plot_candidate_nonbvc

In [ ]:
for which_neuron in plot_candidate_nonbvc:
    plt.figure(figsize=(20,5))
    plt.subplot(1,3,1)
    which_map = place_map_all_all[0][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)

    plt.subplot(1,3,2)
    which_map = place_map_all_all[1][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)
    
    plt.subplot(1,3,3)
    which_map = place_map_all_all[2][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)
    
    
    plt.figure(figsize=(20,5))
    plt.subplot(1,3,1)
    which_map = predict_map_all_all[0][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)

    plt.subplot(1,3,2)
    which_map = predict_map_all_all[1][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)
    
    plt.subplot(1,3,3)
    which_map = predict_map_all_all[2][which_neuron, :,:];
    plt.imshow(which_map)
    plt.title(which_neuron)

In [ ]:
hf = h5py.File(os.path.join(data_path_1, "BVC_tel.h5"), 'r+')
hf.create_dataset('pass_neurons_1', data=pass_neurons_1)
hf.create_dataset('pass_neurons_2', data=pass_neurons_2)
hf.create_dataset('plot_candidate', data=plot_candidate)
hf.create_dataset('plot_candidate_map_1', data=place_map_all_all[0][plot_candidate])
hf.create_dataset('plot_candidate_map_2', data=place_map_all_all[1][plot_candidate])
hf.create_dataset('plot_candidate_map_3', data=place_map_all_all[2][plot_candidate])
hf.create_dataset('plot_candidate_map_1_predict', data=predict_map_all_all[0][plot_candidate])
hf.create_dataset('plot_candidate_map_2_predict', data=predict_map_all_all[1][plot_candidate])
hf.create_dataset('plot_candidate_map_3_predict', data=predict_map_all_all[2][plot_candidate])
hf.create_dataset('plot_candidate_map_12_predict', data=predict_map_12_all[plot_candidate])
hf.create_dataset('corr_diff', data=corr_diff)
hf.close()

In [ ]:
hf = h5py.File(os.path.join(data_path_1, "BVC_tel.h5"), 'r+')
hf.create_dataset('plot_candidate_nonbvc', data=plot_candidate_nonbvc)
hf.create_dataset('plot_candidate_nonbvc_map_1', data=place_map_all_all[0][plot_candidate_nonbvc])
hf.create_dataset('plot_candidate_nonbvc_map_2', data=place_map_all_all[1][plot_candidate_nonbvc])
hf.create_dataset('plot_candidate_nonbvc_map_3', data=place_map_all_all[2][plot_candidate_nonbvc])
hf.create_dataset('plot_candidate_nonbvc_map_1_predict', data=predict_map_all_all[0][plot_candidate_nonbvc])
hf.create_dataset('plot_candidate_nonbvc_map_2_predict', data=predict_map_all_all[1][plot_candidate_nonbvc])
hf.create_dataset('plot_candidate_nonbvc_map_3_predict', data=predict_map_all_all[2][plot_candidate_nonbvc])
hf.create_dataset('plot_candidate_nonbvc_map_12_predict', data=predict_map_12_all[plot_candidate_nonbvc])
hf.create_dataset('corr_all', data=corr_all_all[1])
hf.close()